In [33]:

%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

from IPython.display import clear_output

import os
import pandas as pd
from pathlib import Path
from sklearn import metrics
import statistics
import copy
import networkx as nx

import infrarisk.src.network_sim_models.interdependencies as interdependencies
import infrarisk.src.network_sim_models.power.power_system_model as power
from infrarisk.src.network_sim_models.integrated_network import *

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.cluster import KMeans

from selenium.webdriver import Chrome, ChromeOptions
options = ChromeOptions()
options.add_argument('--headless')

web_driver = Chrome(executable_path='C:/Users/srijith/Downloads/chromedriver_win32/chromedriver.exe', options = options)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
network_dir = Path('../../data/networks/micropolis')
water_folder = network_dir/'water'
power_folder = network_dir/'power'
transpo_folder = network_dir/'transportation'

micropolis_network = IntegratedNetwork(name = 'Micropolis', 
                                       water_folder= water_folder,
                                       power_folder = power_folder,
                                       transp_folder=transpo_folder,
                                       water_sim_type = 'PDA',
                                       power_sim_type='1ph')

Water network successfully loaded from ..\..\data\networks\micropolis\water/water.inp. The analysis type is set to PDA.
initial simulation duration: 60s; hydraulic time step: 60s; pattern time step: 3600s

Power system successfully loaded from ..\..\data\networks\micropolis\power\power.json. Single phase power flow simulation will be used.

Transportation network successfully loaded from ..\..\data\networks\micropolis\transportation. Static traffic assignment method will be used to calculate travel times.
Updating traffic model based on current network conditions...


In [35]:
G_power = micropolis_network.generate_power_networkx_graph()
G_water = micropolis_network.generate_water_networkx_graph()
G_transpo = micropolis_network.generate_transpo_networkx_graph()

In [36]:
for graph in [G_power, G_water, G_transpo]:
    for index, link in enumerate(graph.edges.keys()):
        start_node, end_node = link
        start_coords = graph.nodes[link[0]]['coord']
        end_coords = graph.nodes[link[1]]['coord']
        graph.edges[link]["length"] = round(math.sqrt(((start_coords[0]-end_coords[0])**2)+((start_coords[1]-end_coords[1])**2)),3)

for graph in [G_power, G_water, G_transpo]:
    degree_centrality = nx.degree_centrality(graph)
    node_betweenness_centrality = nx.betweenness_centrality(graph, k=None, normalized=True, weight='length')
    edge_betweenness_centrality = nx.edge_betweenness_centrality(graph, k=None, normalized=True, weight='length')
    eigenvector_centrality = nx.eigenvector_centrality_numpy(graph, max_iter=500, weight='length')
    katz_centrality = nx.katz_centrality_numpy(graph, alpha=0.1, beta=1.0, weight='length')
    closeness_centrality = nx.closeness_centrality(graph, u=None, distance='length', wf_improved=True)
    
    for index, node in enumerate(graph.nodes.keys()):
        graph.nodes[node]["degree_centrality"] = degree_centrality[node]
        graph.nodes[node]["betweenness_centrality"] = node_betweenness_centrality[node]
        graph.nodes[node]["eigenvector_centrality"] = eigenvector_centrality[node]
        graph.nodes[node]["katz_centrality"] = katz_centrality[node]
        graph.nodes[node]["closeness_centrality"] = closeness_centrality[node]
        
    for index, link in enumerate(graph.edges.keys()):
        start_node, end_node = link
        start_coords = graph.nodes[link[0]]['coord']
        end_coords = graph.nodes[link[1]]['coord']
        graph.edges[link]["length"] = round(math.sqrt(((start_coords[0]-end_coords[0])**2)+((start_coords[1]-end_coords[1])**2)),3)
        graph.edges[link]["degree_centrality"] = degree_centrality[start_node] + degree_centrality[end_node]
        graph.edges[link]["betweenness_centrality"] = edge_betweenness_centrality[link]
        graph.edges[link]["eigenvector_centrality"] = eigenvector_centrality[start_node] + eigenvector_centrality[end_node]
        graph.edges[link]["katz_centrality"] = katz_centrality[start_node] + katz_centrality[end_node]
        graph.edges[link]["closeness_centrality"] = closeness_centrality[start_node] + closeness_centrality[end_node]
        

In [37]:
for graph in [G_power, G_water, G_transpo]:
    for index, link in enumerate(graph.edges.keys()):
        link_id = graph.edges[link]['id']
        if link_id.startswith('P_L'):
            power_dict = power.get_power_dict()
            capacity_ref = power_dict['L']["results"]
            capacity_fields = power_dict['L']["capacity_fields"]
            base_pn = micropolis_network.base_power_supply
            base_pn_table = base_pn['line']
            compon_index = base_pn_table[base_pn_table["name"] == link_id].index.item()
            graph.edges[link]['maxflow'] = sum([abs(base_pn[capacity_ref][x][compon_index]) for x in capacity_fields])
        elif link_id.startswith("W_PMA"):
            graph.edges[link]['maxflow'] = micropolis_network.base_water_link_flow[link_id].abs().max().item()
        elif link_id.startswith('T_L'):
            base_tn_dict = getattr(micropolis_network.base_transpo_flow, "link")
            capacity = base_tn_dict[link_id].flow
            graph.edges[link]['maxflow'] = capacity

In [38]:
start_n = 2
end_n = 30
def calculate_wcss(data):
        wcss = []
        for n in range(start_n, end_n + 1):
            kmeans = KMeans(n_clusters=n)
            kmeans.fit(X=data)
            wcss.append(kmeans.inertia_)
    
        return wcss

def optimal_number_of_clusters(wcss):
    x1, y1 = start_n, wcss[0]
    x2, y2 = end_n, wcss[len(wcss)-1]

    distances = []
    for i in range(len(wcss)):
        x0 = i+start_n
        y0 = wcss[i]
        numerator = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
        denominator = math.sqrt((y2 - y1)**2 + (x2 - x1)**2)
        distances.append(numerator/denominator)

    return distances.index(max(distances)) + 2

In [56]:
graph_dict = {"power": G_power, "water": G_water, "transpo": G_transpo}
compon_dict = {"power": 'P_L', "water": 'W_PMA', "transpo": 'T_L'}
cluster_count = {"power": 0, "water": 0, "transpo": 0}
final_df = None

for infra in graph_dict.keys():
    graph = graph_dict[infra]

    edge_df = nx.to_pandas_edgelist(graph)
    edge_df = edge_df[edge_df['id'].str.startswith(compon_dict[infra])]

    #features = edge_df[['degree_centrality', 'closeness_centrality', 'eigenvector_centrality', 'katz_centrality','betweenness_centrality']]
    features = edge_df[['degree_centrality', 'closeness_centrality', 'eigenvector_centrality', 'betweenness_centrality', 'maxflow']]

    features = features.dropna()
    features=(features-features.min())/(features.max()-features.min())
    features = np.array(features)
    sum_of_squares = calculate_wcss(features)
    n = optimal_number_of_clusters(sum_of_squares)
    kmeans = KMeans(n_clusters=n)
    clusters = kmeans.fit_predict(features)
    print(n, edge_df.shape)
    
    cluster_count[infra] = n

    edge_df["cluster"] = [f'{infra}' + str(cluster) for cluster in clusters]
    edge_df["cluster_num"] = list(clusters)
    
    for link in graph.edges.keys():
        id = graph.edges[link]['id']
        if id in edge_df['id'].tolist():
            graph.edges[link]['cluster_num'] = edge_df[edge_df["id"] == id]["cluster_num"].item()
        else:
            graph.edges[link]['cluster_num'] = np.nan
    
    graph_dict[infra] = graph
    
    if final_df is None:
        final_df = edge_df
    else:
        final_df = final_df.append(edge_df, ignore_index=True)

8 (378, 13)
7 (678, 13)
9 (105, 13)


In [57]:
print(final_df.shape)
final_df.to_csv(f"{network_dir}/link_criticality_features.csv", sep = ",", index = False)

(1161, 14)


In [58]:
from bokeh.plotting import figure
from bokeh.transform import factor_cmap, linear_cmap
import bokeh.palettes as palettes
from bokeh.io import show
from bokeh.models import ColumnDataSource, HoverTool,  ColorBar, CategoricalColorMapper
from bokeh.tile_providers import get_provider, Vendors
from bokeh.layouts import gridplot

In [72]:
# [(graph_dict['water'].edges[link]['id'],graph_dict['water'].edges[link]['cluster_num']) for link in graph_dict['water'].edges.keys() if graph_dict['water'].edges[link]['cluster_num'] is not np.nan]

In [74]:
for infra in ["water", "power", "transpo"]:
    p = figure(
        background_fill_color="white",
        plot_width=800,
        height=500,
        #title=f"Component failure probability due to Micropolis floods",
        x_range=(1000, 8000),
        y_range=(1200, 6800),
    )
    
    n = cluster_count[infra]
    graph = graph_dict[infra]
    # print([graph.edges[link]["cluster_num"] for link in graph.edges.keys()])
    palette = getattr(palettes,f"Category10_{n}")

    # links
    x, y, cluster, id = [], [], [], []
    for _, link in enumerate(graph.edges.keys()):
        id_link = graph.edges[link]["id"]
        if id_link in final_df['id'].tolist():
            x.append([graph.nodes[link[0]]["coord"][0], graph.nodes[link[1]]["coord"][0]])
            y.append([graph.nodes[link[0]]["coord"][1], graph.nodes[link[1]]["coord"][1]])
            cluster.append(graph.edges[link]["cluster_num"])
            id.append(id_link)
    print(cluster)
        
    color_mapper = linear_cmap(field_name = "cluster", palette = palette, low = 0, high = n, nan_color = 'black')
    #color_mapper = CategoricalColorMapper(factors=['hi', 'lo'], palette=[RdBu3[2], RdBu3[0]])

    plot_links = p.multi_line(
        "x",
        "y",
        source=ColumnDataSource(
            dict( x=x, y=y, cluster=cluster, id=id)
        ),
        line_color=color_mapper, line_alpha=1, line_width=2.5, 
        #legend_field="fail_prob",
    )
    p.legend.label_text_font_size = '14pt'

    color_bar = ColorBar(color_mapper=color_mapper['transform'], width=20,  location=(0,0),title="Cluster",
                        title_text_font = 'helvetica', title_text_font_style = 'normal', title_text_font_size = '14pt',
                        major_label_text_font_size = '14pt')
    p.add_layout(color_bar, 'right')

    p.legend.location = "bottom_left"
    p.axis.visible = False
    p.grid.visible = False
    p.outline_line_color = None
    show(p)
    
    from bokeh.io import export_png
    export_png(p, filename = f"plots/{infra}_clusters.png", webdriver=web_driver)

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 2, 6, 6, 6, 6, 6, 6, 5, 5, 5, 1, 1, 1, 1, 1, 1, 5, 5, 3, 1, 1, 1, 1, 1, 1, 6, 6, 1, 3, 1, 3, 1, 1, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 5, 1, 3, 1, 1, 1, 5, 5, 1, 1, 1, 1, 3, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 1, 1, 1, 5, 1, 1, 1, 1, 2, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 5, 1, 5, 5, 5, 2, 1, 1, 5, 5, 5, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 6, 6, 6, 6, 6, 6, 6, 1, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 6, 6, 1, 1, 1, 1, 5, 5, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 6, 6, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [12]:
edge_df

,source,target,id,link_type,betweenness_centrality,katz_centrality,maxflow,length,link_category,degree_centrality,eigenvector_centrality,closeness_centrality,cluster,cluster_num
0,T_J1,T_J2,T_L106,Transportation,0.057946,-0.000911,1580.000000,2007.000,Road link,0.089552,1.004341,0.000540,transpo9,9
1,T_J1,T_J60,T_L151,Transportation,0.029412,-0.022883,175.000000,1780.004,Road link,0.044776,0.817525,0.000355,transpo7,7
2,T_J2,T_J31,T_L52,Transportation,0.094381,0.000375,1993.916560,399.362,Road link,0.119403,0.570315,0.000666,transpo9,9
3,T_J2,T_J3,T_L107,Transportation,0.033363,-0.042065,645.000000,384.001,Road link,0.119403,0.631022,0.000699,transpo8,8
4,T_J2,T_J57,T_L158,Transportation,0.076383,0.026104,1665.000000,1201.020,Road link,0.104478,0.789478,0.000649,transpo9,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,T_J48,T_J47,T_L102,Transportation,0.010975,-0.143203,228.647979,368.049,Road link,0.074627,0.000386,0.000500,transpo1,1
101,T_J48,T_J49,T_L103,Transportation,0.037313,-0.065706,330.000000,411.011,Road link,0.104478,0.000958,0.000502,transpo5,5
102,T_J49,T_J67,T_L204,Transportation,0.029412,-0.030866,1830.000000,504.000,Road link,0.074627,0.000733,0.000472,transpo3,3
103,T_J49,T_J50,T_L101,Transportation,0.015364,-0.044888,220.000000,406.100,Road link,0.104478,0.001639,0.000492,transpo5,5
